In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

## Creating a list of Supreme Court Opinions and their URLs
I found that caselaw has a repository of Supreme Court ruling opinions. This notebook contains the code I wrote to compile the list of these URLs and titles to use to grab the opinions.

If you want the data I used for this project, you can start by running these cells. 

In [2]:
root_url = "http://caselaw.findlaw.com/court/us-supreme-court/years/"

In [24]:
years = [year for year in range(2016,2017)]
soupurls = []
souptitles = []
yearlist = []
for year in years:
    search = root_url + str(year)
    response = requests.get(search)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    souped = soup.findAll(class_ = "srpcaselawtr")
    souped = str(souped)
    #souped = souped.findAll("td", {"valign" : "top"})
    souplist = soup.findAll("a")
    for i in souplist:
        if re.search("us-supreme-court", str(i)) and not re.search("years", str(i)) and not re.search("/court/", str(i)): 
            yearlist.append(year)
            soupurls.append(i["href"])
            souptitles.append(i["title"])


In [135]:
supcourt = pd.DataFrame(souptitles,soupurls).reset_index()
supcourt.columns = [["caseurl", "casetitle"]]

def cleanerup(text):
    text = text.replace("<EM>", "")
    text = text.replace("<SPAN STYLE=", "")
    text = text.replace("</EM>", " ")
    return text
supcourt["casetitle"] = supcourt.casetitle.apply(cleanerup)
supcourt["years"] = pd.Series(yearlist)

In [5]:
supcourt.to_pickle("supcourt_yearlist.pickle")

In [9]:
supcourt.ix[20607, "caseurl"]

'http://caselaw.findlaw.com/us-supreme-court/15-338.html'

In [10]:
supcourt.casetitle.to_csv("casetitles.csv") # this was just for creating a stoplist for prepocessing 

In [11]:
supcourt.shape

(23268, 3)